In [ ]:
import pandas as pd
#url = '/content/drive/MyDrive/Dados/dataset-telecon.json'
uurl = 'https://caelum-online-public.s3.amazonaws.com/2929-pandas/dataset-telecon.json'

In [ ]:
dados = pd.read_json(url)
dados

In [ ]:
pd.json_normalize(dados['conta']).head()

In [ ]:
pd.json_normalize(dados['telefone']).head()

In [ ]:
import json

In [ ]:
with open(url) as f:
  json_bruto = json.load(f)

In [ ]:
json_bruto

In [ ]:
dados_normalizados = pd.json_normalize(json_bruto)
dados_normalizados.head()

In [ ]:
dados_normalizados.info()

In [ ]:
#há uma relação dos espaços vazios dentro do DF, realizar multiplicação
idx = dados_normalizados[dados_normalizados['conta.cobranca.Total'] == ' '].index
idx

In [ ]:
#24 é o total de meses
dados_normalizados.loc[idx, 'cliente.tempo_servico'] = 24
dados_normalizados.loc[idx, 'conta.cobranca.Total'] = dados_normalizados.loc[idx, 'conta.cobranca.mensal'] * dados_normalizados.loc[idx, 'cliente.tempo_servico']
dados_normalizados.head()

In [ ]:
dados_normalizados.loc[idx][['cliente.tempo_servico', 'conta.contrato','conta.cobranca.mensal','conta.cobranca.Total']]

In [ ]:
dados_normalizados.info()

In [ ]:
dados_normalizados['conta.cobranca.Total'] = dados_normalizados['conta.cobranca.Total'].astype(float)

In [ ]:
for col in dados_normalizados.columns:
  print(f"Coluna: {col}")
  print(dados_normalizados[col].unique())
  print('-' *30)

In [ ]:
#Query tem que colocar aspas duplas
dados_normalizados.query("Churn == ''")

In [ ]:
dados_sem_vazio = dados_normalizados[dados_normalizados['Churn'] != ''].copy()
dados_sem_vazio.info()

In [ ]:
dados_sem_vazio.reset_index(drop = True, inplace = True)

In [ ]:
#Verificando se há dados duplicados
dados_sem_vazio.duplicated().sum()

In [ ]:
filtro_duplicadas = dados_sem_vazio.duplicated()
filtro_duplicadas

In [ ]:
#Localizando as amostras duplicadas
dados_sem_vazio[filtro_duplicadas]

In [ ]:
#Necessário retirar amostras duplicadas
#Vies do modelo
#Melhora de desempenho
#Aumento da qualidade do modelo
dados_sem_vazio.drop_duplicates(inplace = True) #inplace já faz a alteração direto no código sem precisar fazer atribuição

In [ ]:
dados_sem_vazio.reset_index()

In [ ]:
#Outro problema, dados nulos
dados_sem_vazio.isna()

In [ ]:
dados_sem_vazio.isna().sum()

In [ ]:
#Quandos dados nulos há na base de dados, total considerando linha e coluna
dados_sem_vazio.isna().sum().sum()

In [ ]:
#Quantos index nulos há no dataframe, considerando somente uma única linha
dados_sem_vazio[dados_sem_vazio.isna().any(axis = 1)]
#Axis = 1 pelo menos 1 valor nulo em uma das colunas

In [ ]:
#O que fazer?
#Remover ou preencher?

filtro = dados_sem_vazio['cliente.tempo_servico'].isna()

In [ ]:
dados_sem_vazio[filtro][['cliente.tempo_servico', 'conta.cobranca.mensal', 'conta.cobranca.Total']]

In [ ]:
import numpy as np

In [ ]:
dados_sem_vazio['cliente.tempo_servico'].fillna(
    np.ceil(
        dados_sem_vazio['conta.cobranca.Total'] / dados_sem_vazio['conta.cobranca.mensal']), inplace = True)

In [ ]:
dados_sem_vazio[filtro][['cliente.tempo_servico', 'conta.cobranca.mensal', 'conta.cobranca.Total']]

In [ ]:
dados_sem_vazio.isna().sum()

In [ ]:
#verificando os dados mais frequentes
dados_sem_vazio['conta.contrato'].value_counts()

In [ ]:
colunas_dropar = ['conta.contrato', 'conta.faturamente_eletronico','conta.metodo_pagamento']

In [ ]:
dados_sem_vazio[colunas_dropar].isna().any(axis = 1).sum()

In [ ]:
df_sem_nulos = dados_sem_vazio.dropna(subset = colunas_dropar).copy()
df_sem_nulos.head()

In [ ]:
df_sem_nulos.reset_index(drop=True, inplace=True)

In [ ]:
df_sem_nulos.isna().sum()

In [ ]:
df_sem_nulos.describe()

In [ ]:
#Q1 - 25%
#Q2 - 50% mediana
#Q3 - 75%

import seaborn as sns

In [ ]:
sns.boxplot(x=df_sem_nulos['cliente.tempo_servico'])

In [ ]:
Q1 = df_sem_nulos['cliente.tempo_servico'].quantile(0.25)
Q3 = df_sem_nulos['cliente.tempo_servico'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

In [ ]:
outliers_index = (df_sem_nulos['cliente.tempo_servico'] < limite_inferior) | (df_sem_nulos['cliente.tempo_servico'] > limite_superior)
outliers_index

In [ ]:
df_sem_nulos[outliers_index]['cliente.tempo_servico']

In [ ]:
#substituindo os valores para os outliers
df_sem_out = df_sem_nulos.copy()

In [ ]:
df_sem_out.loc[outliers_index, 'cliente.tempo_servico'] = np.ceil(df_sem_out.loc[outliers_index, 'conta.cobranca.Total'] / df_sem_out.loc[outliers_index, 'conta.cobranca.mensal'])

In [ ]:
sns.boxplot(x=df_sem_out['cliente.tempo_servico'])

In [ ]:
df_sem_out[outliers_index][['cliente.tempo_servico', 'conta.cobranca.mensal', 'conta.cobranca.Total']]

In [ ]:
Q1 = df_sem_out['cliente.tempo_servico'].quantile(0.25)
Q3 = df_sem_out['cliente.tempo_servico'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

outliers_index = (df_sem_out['cliente.tempo_servico'] < limite_inferior) | (df_sem_out['cliente.tempo_servico'] > limite_superior)

In [ ]:
df_sem_out[outliers_index][['cliente.tempo_servico', 'conta.cobranca.mensal', 'conta.cobranca.Total']]

In [ ]:
#Descartando as amostras outliers
#~é uma negação, pega tudo menos
df_sem_out = df_sem_out[~outliers_index]
df_sem_out

In [ ]:
sns.boxplot(x=df_sem_out['cliente.tempo_servico'])

In [ ]:
df_sem_out.reset_index(drop=True, inplace=True)

In [ ]:
#Variáveis categóricas não possuem valores numéricos
df_sem_out

In [ ]:
df_sem_id = df_sem_out.drop('id_cliente', axis=1).copy()

In [ ]:
mapeamento = {
    'nao' : 0,
    'sim' : 1,
    'masculino' : 0,
    'feminino' : 1
}
mapeamento

In [ ]:
for col in df_sem_id.columns:
  print(f"Coluna: {col}")
  print(df_sem_id[col].unique())
  print('-' *30)

In [ ]:
colunas = ['telefone.servico_telefone', 'Churn', 'cliente.parceiro', 'cliente.dependentes', 'conta.faturamente_eletronico', 'cliente.genero']

In [ ]:
df_sem_id[colunas] = df_sem_id[colunas].replace(mapeamento)
df_sem_id

In [ ]:
for col in df_sem_id.columns:
  print(f"Coluna: {col}")
  print(df_sem_id[col].unique())
  print('-' *30)


In [ ]:
#One Hot Encoder (dummy)
s = pd.Series(list('abca'))
s

In [ ]:
#Faz uma matriz de verificação por categoria para verificar onde é 1 ou 0 da categoria listada criando novas colunas por categoria
pd.get_dummies(s, dtype=int)

In [ ]:
df_sem_id.info()

In [ ]:
df_dummies = pd.get_dummies(df_sem_id, dtype=int).copy()

In [ ]:
df_dummies.columns

In [ ]:
df_dummies.info()